In [2]:
import os
import sys
import pandas as pd
from dotmap import DotMap

sys.path.append('../../')

from utils.io import recursive_scan2df, load_yaml
from utils.sampling import ConsecutiveSequences

/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
server = 'local'
server = DotMap(load_yaml('../../config/servers.yml')[server])

database_name = 'cholec80_splits'
database_df = recursive_scan2df(os.path.join(server.database.location, database_name), postfix='.mp4')
database_df

/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,folder
0,split_0.mp4,video36
1,split_1.mp4,video36
2,split_2.mp4,video36
3,split_3.mp4,video36
4,split_4.mp4,video36
...,...,...
3030,split_30.mp4,video35
3031,split_31.mp4,video35
3032,split_32.mp4,video35
3033,split_33.mp4,video35


In [3]:
# individual database post-processing
condition_df = pd.read_csv('20_10_endoscopic_datasets - cholec80_splits.csv')

database_df_ext = database_df.copy()
database_df_ext['vid_idx'] = database_df.folder.apply(lambda x: int(x.replace('video', '')))
database_df_ext['split_idx'] = database_df.file.apply(lambda x: int(x.split('_')[-1].split('.')[0]))

# keep videos to use and discard splits where [change of zoom] or [shift] or [theater] or [status indicator]
init_len = len(database_df_ext)
for _, row in condition_df.iterrows():
    if row.use == False:
        database_df_ext = database_df_ext.drop(database_df_ext[database_df_ext.vid_idx == row.video].index)
    else:
        if not condition_df['change of zoom / split'][condition_df.video == row.video].isna().values:
            idcs = [int(xi) for xi in condition_df['change of zoom / split'][condition_df.video == row.video].values[0].split(',')]
            splits = ~database_df_ext[database_df_ext.vid_idx == row.video].split_idx.isin(idcs)
            splits = splits[splits == False]
            database_df_ext = database_df_ext.drop(splits.index)
        if not condition_df['shift / split'][condition_df.video == row.video].isna().values:
            idcs = [int(xi) for xi in condition_df['shift / split'][condition_df.video == row.video].values[0].split(',')]
            splits = ~database_df_ext[database_df_ext.vid_idx == row.video].split_idx.isin(idcs)
            splits = splits[splits == False]
            database_df_ext = database_df_ext.drop(splits.index)
        if not condition_df['theater / split'][condition_df.video == row.video].isna().values:
            idcs = [int(xi) for xi in condition_df['theater / split'][condition_df.video == row.video].values[0].split(',')]
            splits = ~database_df_ext[database_df_ext.vid_idx == row.video].split_idx.isin(idcs)
            splits = splits[splits == False]
            database_df_ext = database_df_ext.drop(splits.index)
        if not condition_df['status indicator / split'][condition_df.video == row.video].isna().values:
            idcs = [int(xi) for xi in condition_df['status indicator / split'][condition_df.video == row.video].values[0].split(',')]
            splits = ~database_df_ext[database_df_ext.vid_idx == row.video].split_idx.isin(idcs)
            splits = splits[splits == False]
            database_df_ext = database_df_ext.drop(splits.index)
print('Removed {} splits. Left with {}/{} splits'.format(init_len - len(database_df_ext), len(database_df_ext), init_len))
database_df = database_df_ext
database_df_ext


Removed 892 splits. Left with 2143/3035 splits


,file,folder,vid_idx,split_idx
0,split_0.mp4,video36,36,0
1,split_1.mp4,video36,36,1
2,split_2.mp4,video36,36,2
3,split_3.mp4,video36,36,3
4,split_4.mp4,video36,36,4
...,...,...,...,...
3028,split_28.mp4,video35,35,28
3031,split_31.mp4,video35,35,31
3032,split_32.mp4,video35,35,32
3033,split_33.mp4,video35,35,33


In [4]:
p = 0.2
N = int(p*len(database_df))

database_df['train'] = True
database_df_sample = database_df.sample(N)

print(len(database_df_sample)/len(database_df))
database_df_sample.train = False

database_df.update(database_df_sample)

n1 = len(database_df[database_df.train == False])
n2 = len(database_df[database_df.train == True])

database_df = database_df.groupby('vid_idx').apply(lambda dfg: dfg.sample(3)).reset_index(drop=True)
database_df.to_pickle('cholec80_cleaned_samples.pkl')


0.1997200186654223


In [4]:
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from torchvision.transforms import Resize

from datasets import VideoDataset
from utils.io import load_pickle, save_pickle
from utils.processing import endoscopy

visualize = True

database_df = pd.read_pickle('cholec80_cleaned_samples.pkl')
# database_df = database_df.groupby('vid_idx').apply(lambda dfg: dfg.sample(3)).reset_index(drop=True)
# database_df = database_df[database_df.train == False]
# database_df = database_df.iloc[78:]

video_paths = [os.path.join(server.database.location, database_name, x.folder, x.file) for _, x in database_df.iterrows()]
frame_rate = 5
duration = 10
clip_length_in_frames = int(frame_rate*duration)
num_workers = 8

# load metadata, if existing
metadata_name = 'cholec80_splits_metadata.pkl' # -> 'cholec80_splits_metadata_full.pkl'
metadata = None
print('\nLooking for {}'.format(metadata_name))
if os.path.exists(metadata_name):
    metadata = load_pickle(metadata_name)
    print('Found.')
else:
    print('Could not find {}. Generating metadata.'.format(metadata_name))

video_ds = VideoDataset(
            video_paths=video_paths,
            clip_length_in_frames=clip_length_in_frames,
            frames_between_clips=clip_length_in_frames,
            frame_rate=frame_rate,
            precomputed_metadata=metadata,
            num_workers=num_workers,
            permute=False,
            convert_dtype=False
)

# store metadata
if not os.path.exists(metadata_name):
    metadata = video_ds.metadata
    save_pickle(metadata_name, metadata)

# prepare database
database_transforms = pd.DataFrame(
    columns=['database', 'train', 'file', 'pre_transforms', 'aug_transforms', 'auxiliary']
)

# boundary detection parameters
illumination_th = 0.998
detected_video_idx = -1


# iterate through each video
for batch in video_ds:
    video, augmented_video, frame_rate, video_fps, video_idx, idx = batch
    if video_idx == detected_video_idx:
        continue
    video_path = database_df.iloc[video_idx].folder
    video_name = database_df.iloc[video_idx].file

    for frame in video:
        frame = frame.numpy()
        paddding = 5
        frame = frame[paddding:-paddding] # add to crop
        
        # pre-process frame
        mask = endoscopy.bilateralSegmentation(frame, th=0.05)
        center, radius = endoscopy.boundary_detection.ransacBoundaryCircle(mask, fit='numeric', n_iter=1, n_pts=100)

        if radius is not None:
            illumination = endoscopy.illuminationLevel(mask, center, radius)

            print('\rIllumination level: {}, threshold: {}, video {} - {}/{}, idx {}/{}, shape: {}'.format(illumination, illumination > illumination_th, video_idx+1, video_path, video_name, idx+1, len(video_ds), video.shape), end='')

            top_left, shape = endoscopy.maxRectangleInCircle(frame.shape, center, radius)
            center, radius = center.astype(int), int(radius)
            top_left, shape = top_left.astype(int), tuple(map(int, shape))

            if illumination > illumination_th:
                if visualize:
                    cv2.circle(frame, (center[1], center[0]), radius, (255, 255, 0), 2)
                    cv2.rectangle(frame, (top_left[1], top_left[0]), (top_left[1] + shape[1], top_left[0] + shape[0]), (255, 255, 0), 2)

                    cv2.imshow('img', frame[...,::-1])
                    cv2.imshow('mask', mask)
                    cv2.waitKey()

                detected_video_idx = video_idx

                inp = input()
                if inp == 'y':
                    row = {
                        'database': 'cholec80_splits',
                        'train': database_df.iloc[video_idx].train,
                        'file': {
                            'name': video_name,
                            'path': video_path
                        },
                        'pre_transforms': [
                            {
                                'module': 'utils.transforms',
                                'type': 'Crop',
                                'kwargs': {
                                    'top_left_corner': [top_left[0]+paddding, top_left[1]],
                                    'shape': [shape[0]+paddding, shape[1]],
                                    'order': 'chw'
                                }
                            },
                            {
                                'module': 'torchvision.transforms',
                                'type': 'Resize',
                                'kwargs': {
                                    'size': [240, 320]
                                }
                            }
                        ],
                        'aug_transforms': [],
                        'auxiliary': {}
                    }

                database_transforms = database_transforms.append(row, ignore_index=True)
                database_transforms.to_pickle('cholec80_splits_transforms.pkl')
                break
        else:
            print('\rRadius not found.', end='')

# database_transforms.to_pickle('cholec80_splits_transforms.pkl')
if visualize:
    cv2.destroyAllWindows()

/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "



Looking for cholec80_splits_metadata.pkl
Found.
Illumination level: 0.9994102301005316, threshold: True, video 1 - video01/split_3.mp4, idx 1/1170, shape: torch.Size([50, 480, 854, 3])

In [8]:
# create test set
database_transforms = pd.read_pickle('cholec80_splits_transforms.pkl')

p = 0.2
N = int(p*len(database_transforms))

database_transforms['train'] = True
database_transforms_sample = database_transforms.sample(N)

database_transforms_sample.train = False

database_transforms.update(database_transforms_sample)

n1 = len(database_transforms[database_transforms.train == False])
n2 = len(database_transforms[database_transforms.train == True])
print(n1/(n1+n2))

database_transforms.to_pickle('cholec80_splits_transforms.pkl')

0.19969666329625885


In [3]:
import os
import pandas as pd
from dotmap import DotMap

from utils.io import load_yaml
from utils.io import load_pickle, save_pickle
from utils.transforms import anyDictListToCompose
from lightning_data_modules import VideoDataModule


meta_df_name = 'cholec80_splits_transforms.pkl'
meta_df = pd.read_pickle(meta_df_name)
meta_df.aug_transforms = None  # no transforms for evaluation

test_md_name = 'cholec80_splits_test_md.pkl'
test_md = None
print('\nLooking for {}'.format(test_md_name))
if os.path.exists(test_md_name):
    test_md = load_pickle(test_md_name)
    print('Found.')
else:
    print('Could not find {}. Generating metadata.'.format(test_md_name))

# load cholec80 splits
prefix = server.database.location
clip_length_in_frames = 10
frames_between_clips = clip_length_in_frames
frame_rate = 5
train_split = 0.8
batch_size = 1
num_workers = 4
random_state = 42

dm = VideoDataModule(
    meta_df,
    prefix=prefix,
    clip_length_in_frames=clip_length_in_frames,
    frames_between_clips=frames_between_clips   ,
    frame_rate=frame_rate,
    train_split=train_split,
    batch_size=batch_size,
    num_workers=num_workers,
    random_state=random_state,
    test_metadata=test_md
)

_, _, test_md = dm.setup('test')

# store metadata
save_pickle(test_md_name, test_md)



Looking for cholec80_splits_test_md.pkl
Found.


In [4]:
import cv2
from tqdm import tqdm
from kornia import tensor_to_image

test_dl = dm.test_dataloader()

for batch in tqdm(test_dl):
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    for frame in vid[0]: # BxNxCxHxW
        cv2.imshow('frame', tensor_to_image(frame))
        cv2.waitKey(1)

cv2.destroyAllWindows()


  0%|          | 0/11850 [00:00<?, ?it/s]/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


In [1]:
inp = input()

if inp == 'y':
    print('keep')
else:
    print('discard')

discard
